In [ ]:
import gym
import numpy as np
import time


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual, Output, Controller
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
import functools
from jupyter_ui_poll import ui_events

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
#blgd-v0 - BlindGrasp with Discrete actions-v0
#GUT=True - works only locally and not in Google Colab
#ResetCount=250, Reset after 250 actions
env=gym.make("gym_blg:blgd-v0", GUI=True,ResetCount=250)

In [ ]:
env.reset()

In [ ]:
def decode_obs(obs):
    #decodes the observation into sensor readings (normalized)
    #proximity
    # - Proximity sensor array, 22 values, 0 or 1
    # - we are using only using the sensors from the tip - 7 on side and 4 at tip of each fingwe
    #pos
    # - End effector x,y,z position - (stacked with 2 previous positions for temporal info)
    #   9 values - normalized between 0 and 1 
    #force
    # - End Effector x,y,x force - 3 values, normalized between 0-1
    #ObjMap
    # - The workspace(tray) are is divided into 32x32 cells
    # - ObjMap marks those positions/cells where the proximity sensors detected the presence of any object
    # - If object present 1, else 0
    #VisitMap
    # - VisitMap marks the cells (of 32x32 array) visited by end effector as 1.
    # - This info is to incentivize the agent to visit unexplored cells
    #CurPosMap
    # - The cell corresponding to the xy position of end effector is 1 in this 32x32 array
    #GelSight1 and GelSight2
    # - GelSight Depth data from the gripper, 32x32 each
    # - values normalized between 0-1
    prox = obs[:17]
    pos = obs[17:26]
    force = obs[26:29]
    ObjMap= obs[29:1053] #1053 = 29 +(32*32)
    VisitMap = obs[1053:2077] #2077=1053+(32*32)
    CurPosMap =obs[2077:3101] #3101=2077+(32*32)
    #GelSight1=obs[3101:4125] #4125=3101+(32*32)
    #GelSight2=obs[4125:5149]# 5149=4125+(32*32) 
    return prox,pos,force,ObjMap,VisitMap,CurPosMap#,GelSight1, GelSight2
    

In [ ]:
humancmd = None
ui_done = False
#GUI button callback
def on_button_clicked(b, rs_=0):
    #step
    global ui_done
    global humancmd
    humancmd = rs_
    ui_done= True
    '''
    obs, rew,Done,misc  = env.step(rs_)
    #decode observations
    #prox,pos,force,ObjMap,VisitMap,CurPosMap,GelSight1, GelSight2 = decode_obs(obs)
    #remove gelsight
    prox,pos,force,ObjMap,VisitMap,CurPosMap = decode_obs(obs)
   
    #update maps
    #add CurPosMap to ObjMap and VisitMap for more intuitive visualization
    map_id1.set_data((ObjMap.reshape(32,32)+(CurPosMap.reshape(32,32)*10.0)))
    map_id2.set_data((VisitMap.reshape(32,32)+(CurPosMap.reshape(32,32)*10.0)))
    #map_id3.set_data(CurPosMap.reshape(32,32))
    #remove gelsight
    #map_id4.set_data(GelSight1.reshape(32,32))
    #map_id5.set_data(GelSight2.reshape(32,32))
    #TODO -  save the state-action-obs-reward for pretraining the agent
    rBox.value=rew
    pos = np.around(pos,3)
    force = np.around(force,3)
    tBoxVal = str(pos[0])+" "+ str(pos[1])+" "+str(pos[2]) +" F " + str(force[0]) +" "+ str(force[1]) +" "+ str(force[2])
    #print(rew)
    tBox.value=tBoxVal
    '''
    

In [ ]:
# Reset callback from GUI button
def resetenv(b):
    env.reset()

In [ ]:
#define the GUI buttons
buttonl = widgets.Button(description="Left")
buttonr = widgets.Button(description="Right")
buttonf = widgets.Button(description="Fwd")
buttonb = widgets.Button(description="Back")
buttonl.style.button_color = 'lightgreen'
buttonr.style.button_color = 'lightgreen'
buttonf.style.button_color = 'lightgreen'
buttonb.style.button_color = 'lightgreen'

buttonfl = widgets.Button(description="F Left")
buttonfr = widgets.Button(description="F Right")
buttonbl = widgets.Button(description="B Left")
buttonbr = widgets.Button(description="B Right")

buttonu = widgets.Button(description="Up")
buttond = widgets.Button(description="Down")
buttonuj = widgets.Button(description="Up 10x")
buttondj = widgets.Button(description="Down 10x")
buttonu.style.button_color = 'lightblue'
buttond.style.button_color = 'lightblue'
buttonuj.style.button_color = 'lightblue'
buttondj.style.button_color = 'lightblue'


buttongro = widgets.Button(description="Gripper Open")
buttongrc = widgets.Button(description="Gripper Close")
buttongro.style.button_color = 'moccasin'
buttongrc.style.button_color = 'moccasin'

buttonrst = widgets.Button(description="RESET")
buttonrst.style.button_color = 'red'
#Define the callback
buttonl.on_click(functools.partial(on_button_clicked, rs_=4))
buttonr.on_click(functools.partial(on_button_clicked, rs_=5))
buttonf.on_click(functools.partial(on_button_clicked, rs_=6))
buttonb.on_click(functools.partial(on_button_clicked, rs_=7))

buttonfl.on_click(functools.partial(on_button_clicked, rs_=8))
buttonfr.on_click(functools.partial(on_button_clicked, rs_=9))
buttonbl.on_click(functools.partial(on_button_clicked, rs_=10))
buttonbr.on_click(functools.partial(on_button_clicked, rs_=11))

buttonu.on_click(functools.partial(on_button_clicked, rs_=0))
buttond.on_click(functools.partial(on_button_clicked, rs_=1))
buttonuj.on_click(functools.partial(on_button_clicked, rs_=2))
buttondj.on_click(functools.partial(on_button_clicked, rs_=3))

buttongro.on_click(functools.partial(on_button_clicked, rs_=12))
buttongrc.on_click(functools.partial(on_button_clicked, rs_=13))

buttonrst.on_click(resetenv)

rBox=widgets.FloatText(value=2.0, disabled=True)
tBox=widgets.Text(value="X-Y-Z Positions - Forces", disabled=True)

#organize the buttons
col1=widgets.VBox([buttongro,buttonbl,buttonl,buttonfl])
col2=widgets.VBox([buttonb,buttonu,buttond,buttonf])
col3=widgets.VBox([buttongrc,buttonbr,buttonr,buttonfr])
col4=widgets.VBox([buttonuj,buttondj,buttonrst,rBox])
col5=widgets.HBox([tBox])

In [ ]:
#gamepad callbacks
def on_click2(change):
    #print("ev",pad.buttons[0] )
    if(pad.buttons[12].value==1.0):
        #print("F")
        on_button_clicked(None,7)
    elif(pad.buttons[13].value==1.0):
        #print("B")
        on_button_clicked(None,6)
    elif(pad.buttons[14].value==1.0):
        #print("L")
        on_button_clicked(None,4)
    elif(pad.buttons[15].value==1.0):
        #print("R")
        on_button_clicked(None,5)
    elif(pad.buttons[4].value==1.0):
        #print("U")
        on_button_clicked(None,0)
    elif(pad.buttons[6].value==1.0):
        #print("D")
        on_button_clicked(None,1)
    elif(pad.buttons[7].value==1.0):
        #print("DD")
        on_button_clicked(None,3)
    elif(pad.buttons[1].value==1.0):
        #print("O")
        on_button_clicked(None,12)
    elif(pad.buttons[2].value==1.0):
        #print("C")
        on_button_clicked(None,13)
    elif(pad.buttons[3].value==1.0):
        #print("RESET")
        resetenv(None)
        #on_button_clicked(None,13)    

In [ ]:
gamepad=True
if(gamepad):
    pad = Controller()
    pad
#press any buttons on gamepad, else it may not initialize properly

In [ ]:
if(gamepad):

    pad.buttons[12].observe(on_click2, names='value')
    pad.buttons[13].observe(on_click2, names='value')
    pad.buttons[14].observe(on_click2, names='value')
    pad.buttons[15].observe(on_click2, names='value')
    pad.buttons[6].observe(on_click2, names='value')
    pad.buttons[4].observe(on_click2, names='value')
    pad.buttons[7].observe(on_click2, names='value')

    pad.buttons[1].observe(on_click2, names='value')
    pad.buttons[2].observe(on_click2, names='value')
    pad.buttons[3].observe(on_click2, names='value')

In [ ]:
#plots
f = plt.figure(figsize=(4,1.5),dpi= 250)

i1=f.add_subplot(1,4,1)
i1.set_title('ObjMap ')
i1.axis('off')
map_id1=plt.imshow((np.random.rand(32,32)*10)//2)
i2=f.add_subplot(1,4,2)
i2.set_title('VisitMap ')
i2.axis('off')
map_id2=plt.imshow((np.random.rand(32,32)*10)//2)
#i3=f.add_subplot(1,5,3)
#i3.set_title('CurPosMap ')
#i3.axis('off')
#map_id3=plt.imshow((np.random.rand(32,32)*10)//2)
i4=f.add_subplot(1,4,3)
i4.set_title('GelS-1 ')
i4.axis('off')
map_id4=plt.imshow((np.random.rand(32,32)*10)//2)
i5=f.add_subplot(1,4,4)
i5.set_title('GelS-2 ')
i5.axis('off')
map_id5=plt.imshow((np.random.rand(32,32)*10)//2)




In [ ]:
#from stable_baselines import PPO2
#from stable_baselines.gail import ExpertDataset
#TODO - save the action-observation-reward from human demonstration as specified in
# https://stable-baselines.readthedocs.io/en/master/guide/pretrain.html#data-structure-of-the-expert-dataset
#https://stable-baselines.readthedocs.io/en/master/_modules/stable_baselines/gail/dataset/record_expert.html

In [ ]:
#reset the env and get initial observation
obs = env.reset()

In [ ]:
n_episodes =  5 #number of episodes

In [ ]:
actions = []
observations = []
rewards = []
episode_returns = np.zeros((n_episodes,))
episode_starts = []

In [ ]:
ep_idx = 0

episode_starts.append(True)
reward_sum = 0.0
idx = 0

In [ ]:
save_path = "demos.npz"

In [ ]:
def visualize(obs, rew=0):
    prox,pos,force,ObjMap,VisitMap,CurPosMap = decode_obs(obs)
    map_id1.set_data((ObjMap.reshape(32,32)+(CurPosMap.reshape(32,32)*10.0)))
    map_id2.set_data((VisitMap.reshape(32,32)+(CurPosMap.reshape(32,32)*10.0)))
    rBox.value=rew
    pos = np.around(pos,3)
    force = np.around(force,3)
    tBoxVal = str(pos[0])+" "+ str(pos[1])+" "+str(pos[2]) +" F " + str(force[0]) +" "+ str(force[1]) +" "+ str(force[2])
    tBox.value=tBoxVal

In [ ]:
#Display GUI
widgets.HBox([col1,col2,col3,col4])


In [ ]:
#text box to show position and force values
widgets.HBox([col5])

In [ ]:
while ep_idx < n_episodes:
    observations.append(obs)
    
    if(idx==0):
        visualize(obs)
    else:
        visualize(obs,reward)
    #get action from human input
    #events = get_gamepad() 
    #while(checkButton()==None):
    ui_done = False
    with ui_events() as poll:
        while ui_done is False:
            poll(10)
            time.sleep(0.15)
    time.sleep(0.1)
    
    action =humancmd
    humancmd=None
    obs, reward, done, _ = env.step(action)
    visualize(obs, reward)
    
    actions.append(action)
    rewards.append(reward)
    episode_starts.append(done)
    reward_sum += reward
    idx += 1
    #print("gathering samples", idx, ep_idx)
    
    if done:
        obs = env.reset()
        episode_returns[ep_idx] = reward_sum
        reward_sum = 0.0
        ep_idx += 1
    
observations = np.concatenate(observations).reshape((-1,) + env.observation_space.shape)
actions = np.array(actions).reshape((-1, 1))
rewards = np.array(rewards)
episode_starts = np.array(episode_starts[:-1])
    
assert len(observations) == len(actions)
numpy_dict = {
    'actions': actions,
    'obs': observations,
    'rewards': rewards,
    'episode_returns': episode_returns,
    'episode_starts': episode_starts
}# type: Dict[str, np.ndarray]
    
for key, val in numpy_dict.items():
     print(key, val.shape)

if save_path is not None:
    np.savez(save_path, **numpy_dict)

env.close()    
    